# Fortune 500 Companies Database

### This notebook Transforms the following data into fortune500_db.sqlite
- Fortune 500 companies
- Fortune 500 stock values
- Fortune 500 job postings
- Fortune 500 job reviews
- Global Exchange Rates
- United States Population by city

## Import necessary libraries:

In [1]:
import pandas as pd

In [2]:
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

## Create engine and connection to sqlite:

In [3]:
from config import username, passwd, location, database

engine = create_engine(f"mysql://{username}:{passwd}@{location}/{database}")
conn = engine.connect()

## Create declarative base and create session to add to database:

In [4]:
Base = declarative_base()

Base.metadata.create_all(engine)

from sqlalchemy.orm import Session
session = Session(bind=engine)

## Import all csv files into a dataframe:

#### Start with fortune 500 comapnies and proceed with others.

In [5]:
fortune500_path = "./Data/Fortune_500_list_databahn_2018.csv"

In [6]:
fortune500_df = pd.read_csv(fortune500_path, encoding = "ISO-8859-1")
fortune500_df.columns

Index(['Fortune  Rank (2018)', 'Company Name', 'Ticker Symbol',
       'Fortune Rank (2017)', 'Revenues', 'Revenues change', 'Profits',
       'Profit Change', 'Assets', 'Market Value', 'Employee Count', 'CEO Name',
       'CEO Title', 'Sector', 'Industry', 'HQ Location', 'Website URL',
       'Years on Fortune List', 'Fortune URL'],
      dtype='object')

## Edit column headers to be friendlier to SQL.

In [7]:
fortune500_df.rename(columns={
    "Fortune  Rank (2018)":"fortune_rank",
    "Company Name":"name",
    "Ticker Symbol":"ticker",
    "Fortune Rank (2017)":"rank_2017",
    "Revenues":"revenues",
    "Revenues change":"revenue_change",
    "Profits":"profits",
    "Profit Change":"profit_change",
    "Assets":"assets",
    "Market Value":"market_value",
    "Employee Count":"num_employees",
    "CEO Name":"ceo_name",
    "CEO Title":"ceo_title",
    "Sector":"sector",
    "HQ Location":"hq_location",
    "Website URL":"url",
    "Years on Fortune List":"years_on_fortune_list",
    "Fortune URL":"fortune_url"
}, inplace=True)

In [8]:
# Check all rows with \x92,  there is a string error: 
fortune500_df[fortune500_df['name'].str.contains("\x92")==True]

,fortune_rank,name,ticker,rank_2017,revenues,revenue_change,profits,profit_change,assets,market_value,num_employees,ceo_name,ceo_title,sector,Industry,hq_location,url,years_on_fortune_list,fortune_url
39,40.0,Lowes,LOW,40.0,"$68,619",5.5%,"$3,447.0",11.4%,"$35,291","$72,812","255,000",Robert A. Niblock,"Chairman, President & Chief Executive Officer",Retailing,Specialty Retailers: Other,"Mooresville, N.C.",http://www.lowes.com,24.0,http://fortune.com/fortune500/lowes/
119,120.0,Macys,M,110.0,"$24,837",-3.7%,"$1,547.0",149.9%,"$19,381","$9,065","130,000",Jeffrey Gennette,Chairman & Chief Executive Officer,Retailing,General Merchandisers,Cincinnati,http://www.macysinc.com,24.0,http://fortune.com/fortune500/macys/
130,131.0,McDonalds,MCD,112.0,"$22,820",-7.3%,"$5,192.3",10.8%,"$33,804","$124,244","235,000",Stephen J. Easterbrook,"President, Chief Executive Officer & Director","Hotels, Restaurants & Leisure",Food Services,"Oak Brook, Ill.",http://www.aboutmcdonalds.com,24.0,http://fortune.com/fortune500/mcdonalds/
156,157.0,Kohls,KSS,150.0,"$19,095",2.2%,$859.0,54.5%,"$13,340","$11,021","85,000",Michelle D. Gass,Chairman & Chief Executive Officer,Retailing,General Merchandisers,"Menomonee Falls, Wis.",http://www.kohls.com,21.0,http://fortune.com/fortune500/kohls/
215,216.0,Land OLakes,NaN,209.0,"$13,740",3.8%,$314.2,28.3%,"$9,509",NaN,"10,000",Christopher J. Policinski,"President, Chief Executive Officer & Director","Food, Beverages & Tobacco",Food Consumer Products,"Arden Hills, Minn.",http://www.landolakesinc.com,16.0,http://fortune.com/fortune500/land-olakes/
328,329.0,OReilly Automotive,ORLY,323.0,"$8,978",4.5%,"$1,133.8",9.3%,"$7,572","$20,607","60,365",Gregory D. Johnson,"President, Chief Executive Officer & Director",Retailing,Specialty Retailers: Other,"Springfield, Mo.",http://www.oreillyauto.com,9.0,http://fortune.com/fortune500/oreilly-automotive/
338,339.0,Peter Kiewit Sons,NaN,324.0,"$8,678",1.2%,$371.0,-6.3%,"$5,710",NaN,"22,000",Bruce E. Grewcock,"President, Chief Executive Officer & Director",Engineering & Construction,"Engineering, Construction",Omaha,http://www.kiewit.com,19.0,http://fortune.com/fortune500/peter-kiewit-sons/
339,340.0,Dicks Sporting Goods,DKS,340.0,"$8,591",8.4%,$323.4,12.5%,"$4,204","$3,765","30,300",Edward W. Stack,Chairman & Chief Executive Officer,Retailing,Specialty Retailers: Other,"Coraopolis, Pa.",http://www.dicks.com,9.0,http://fortune.com/fortune500/dicks-sporting-g...
424,425.0,Caseys General Stores,CASY,423.0,"$6,641",5.3%,$177.5,-21.5%,"$3,020","$4,120","25,463",Terry W. Handley,"President, Chief Executive Officer & Director",Retailing,Specialty Retailers: Other,"Ankeny, Iowa",http://www.caseys.com,8.0,http://fortune.com/fortune500/caseys-general-s...
438,439.0,Dillards,DDS,417.0,"$6,423",0.1%,$221.3,30.8%,"$3,673","$2,289","31,400",William T. Dillard II,Chairman & Chief Executive Officer,Retailing,General Merchandisers,"Little Rock, Ark.",http://www.dillards.com,24.0,http://fortune.com/fortune500/dillards/


In [9]:
# replace these rows with "". 
fortune500_df["name"].replace({'\x92|\x93|\x94':''}, regex=True, inplace=True)
fortune500_df[fortune500_df['name'].str.contains("\x92")==True]

,fortune_rank,name,ticker,rank_2017,revenues,revenue_change,profits,profit_change,assets,market_value,num_employees,ceo_name,ceo_title,sector,Industry,hq_location,url,years_on_fortune_list,fortune_url


In [10]:
# Check to see if it was fixed:
fortune500_df.at[271,"name"]

'Toys R Us'

## Push DataFrame to sqlite and check to see if written.

In [11]:
fortune500_df.set_index("fortune_rank", inplace=True)
fortune500_df.to_sql("fortune500_companies", conn, if_exists="replace")

Note: If the datatypes in SQL are set, this should be "append" and not "replace".
- Replace will rewrite the data types if they are defined in sql.

In [12]:
f500 = pd.read_sql_query("select * from fortune500_companies", conn)

In [13]:
f500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 19 columns):
fortune_rank             500 non-null float64
name                     502 non-null object
ticker                   471 non-null object
rank_2017                495 non-null float64
revenues                 500 non-null object
revenue_change           497 non-null object
profits                  500 non-null object
profit_change            436 non-null object
assets                   500 non-null object
market_value             472 non-null object
num_employees            500 non-null object
ceo_name                 500 non-null object
ceo_title                500 non-null object
sector                   500 non-null object
Industry                 500 non-null object
hq_location              500 non-null object
url                      500 non-null object
years_on_fortune_list    500 non-null float64
fortune_url              500 non-null object
dtypes: float64(3), object(16)
memory

## Add other datasets to database:

In [14]:
stock_values_path = "./Data/stock_values.csv"

In [15]:
stock_values_df = pd.read_csv(stock_values_path, encoding = "ISO-8859-1")
stock_values_df.head()

,Unnamed: 0,symbol,date,open,high,low,close,volume
0,0,WMT,2018-10-26,98.81,99.670,98.0200,98.94,14925551
1,1,WMT,2018-10-25,98.01,99.480,97.0000,99.18,10737074
2,2,WMT,2018-10-24,97.50,99.170,97.2750,97.56,10364712
3,3,WMT,2018-10-23,96.50,97.985,96.1100,97.80,9405824
4,4,WMT,2018-10-22,97.34,97.580,96.7434,97.14,7122857


### Rename columns for sql: 

In [16]:
stock_values_df.rename(columns={
    "Unnamed: 0":"id",
    "symbol":"ticker"
}, inplace=True)

In [17]:
stock_values_df.set_index("id", inplace=True)
stock_values_df.head()

,ticker,date,open,high,low,close,volume
id,,,,,,,
0,WMT,2018-10-26,98.81,99.670,98.0200,98.94,14925551
1,WMT,2018-10-25,98.01,99.480,97.0000,99.18,10737074
2,WMT,2018-10-24,97.50,99.170,97.2750,97.56,10364712
3,WMT,2018-10-23,96.50,97.985,96.1100,97.80,9405824
4,WMT,2018-10-22,97.34,97.580,96.7434,97.14,7122857


In [18]:
stock_values_df.to_sql("stock_values", conn, if_exists="replace")

## Add Apple Reviews

In [19]:
apple_reviews_path = "./Data/Apple_reviews.csv"
apple_reviews_df = pd.read_csv(apple_reviews_path, encoding = "ISO-8859-1")
apple_reviews_df.head()

,Unnamed: 0,company,rating_number,reviewer,review_job_title,review_job_location,review_text,review_date_created,page_retrieved
0,0,Apple,5.0,Technical/ Sales Specialist,Technical/ Sales SpecialistÂ (Former Employee)...,"Edison, NJ",There is a lot to learn at this company. There...,"September 26, 2018",1
1,0,Apple,3.0,Software Engineer,Software EngineerÂ (Current Employee) â Â,"Cupertino, CA",Your experience at Apple really depends only w...,"October 29, 2018",1
2,0,Apple,5.0,Technical Specialist,Technical SpecialistÂ (Current Employee) â Â,"Northridge, CA",Apple is a company that believes in their cons...,"October 28, 2018",1
3,0,Apple,4.0,Technical Advisor,Technical AdvisorÂ (Current Employee) â Â,At Home,"A typical day at work is an 8-10 hour day, shi...","October 28, 2018",1
4,0,Apple,3.0,Material Handler 3,Material Handler 3Â (Former Employee) â Â,"Elk Grove, CA",I was very excited when I started here. It is ...,"October 28, 2018",1


In [20]:
apple_reviews_df.drop(columns=["Unnamed: 0", "review_job_title", "page_retrieved"], inplace=True)
apple_reviews_df.head()

,company,rating_number,reviewer,review_job_location,review_text,review_date_created
0,Apple,5.0,Technical/ Sales Specialist,"Edison, NJ",There is a lot to learn at this company. There...,"September 26, 2018"
1,Apple,3.0,Software Engineer,"Cupertino, CA",Your experience at Apple really depends only w...,"October 29, 2018"
2,Apple,5.0,Technical Specialist,"Northridge, CA",Apple is a company that believes in their cons...,"October 28, 2018"
3,Apple,4.0,Technical Advisor,At Home,"A typical day at work is an 8-10 hour day, shi...","October 28, 2018"
4,Apple,3.0,Material Handler 3,"Elk Grove, CA",I was very excited when I started here. It is ...,"October 28, 2018"


In [21]:
apple_reviews_df.index.name = 'id'
apple_reviews_df.to_sql("company_reviews", conn, if_exists="replace")

## Add Exchange Rates

In [22]:
exchange_path = "./Data/exchange.csv"
exchange_df = pd.read_csv(exchange_path, encoding = "ISO-8859-1")
exchange_df.head()

,Unnamed: 0,fromcode,fromname,tocode,toname,rate,lastrefreshed
0,0,AFN,Afghan Afghani,USD,United States Dollar,0.013351,2018-10-29 23:24:59
1,1,ARS,Argentine Peso,USD,United States Dollar,0.027115,2018-10-29 23:25:09
2,2,AWG,Aruban Florin,USD,United States Dollar,0.561798,2018-10-29 23:25:20
3,3,AUD,Australian Dollar,USD,United States Dollar,0.706015,2018-10-29 23:25:30
4,4,AZN,Azerbaijani Manat,USD,United States Dollar,0.588408,2018-10-29 23:25:40


In [23]:
exchange_df.rename(columns={
    "Unnamed: 0":"id",
    "fromcode":"from",
    "tocode":"to",
    "lastrefreshed":"last_refreshed"
}, inplace=True)

In [24]:
exchange_df.set_index("id", inplace=True)

In [25]:
exchange_df.to_sql("exchange", conn, if_exists="replace")

## Add Indeed file:
note this is a dat file

In [26]:
indeed_path = "./Data/indeed.dat"
indeed_df = pd.read_csv(indeed_path)

indeed_df.head()

,Unnamed: 0,age,company,desc,jcid,jkid,job_no,loc,query_date,query_time,title
0,0,2 days ago,Walmart,"2 years experience with ETL, data modeling, SQ...",822bc5d9a49270ea,1b40383fd862d403,1,"Bentonville, AR 72712",2018-10-29,20:48:18,Business Intelligence Analyst -LEG
1,1,2 days ago,Walmart,"Fluency in SQL, as well as an understanding of...",822bc5d9a49270ea,27ec2b40b1200fe2,2,"San Bruno, CA 94066",2018-10-29,20:48:18,Product Analytics Manager
2,2,2 days ago,Walmart,Position Description As a Software Engineer II...,822bc5d9a49270ea,a3747a644aa53898,3,"Bentonville, AR 72712",2018-10-29,20:48:18,Software Engineer III
3,3,2 days ago,Walmart,Position Description In this role you will · A...,822bc5d9a49270ea,25492a3d97275785,4,"Sunnyvale, CA",2018-10-29,20:48:18,Search/Relevance - Staff Data Scientist
4,4,2 days ago,Walmart,"Experience in RESTful Services, RDBMS, NoSql, ...",822bc5d9a49270ea,47a8e7b1fc47f6a7,5,"Sunnyvale, CA",2018-10-29,20:48:18,Staff Software Engineer


In [27]:
indeed_df.drop(columns=["query_time","desc"], inplace=True)

In [28]:
indeed_df.rename(columns={
    "Unnamed: 0":"id",
    "jcid":"company_id",
    "jkid":"job_id"
}, inplace=True)

In [29]:
indeed_df.set_index("id", inplace=True)

In [30]:
indeed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2831 entries, 0 to 2830
Data columns (total 8 columns):
age           2831 non-null object
company       2831 non-null object
company_id    2831 non-null object
job_id        2831 non-null object
job_no        2831 non-null int64
loc           2831 non-null object
query_date    2831 non-null object
title         2831 non-null object
dtypes: int64(1), object(7)
memory usage: 199.1+ KB


In [31]:
indeed_df.to_sql("indeed", conn, if_exists="replace")

### Add population data:

In [41]:
population_path = "./Data/Population_Data.csv"
population_df = pd.read_csv(population_path)

population_df.head()

,Rank,Name,State,2018 Population,2016 Population,2010 Census,Change,2018 Density
0,1,New York,New York,8580015,8537673,8175133,0.002474,11029.232132
1,2,Los Angeles,California,4030668,3976322,3792621,0.006787,3320.644797
2,3,Chicago,Illinois,2687682,2704958,2695598,-0.003204,4564.612213
3,4,Houston,Texas,2340814,2303482,2099451,0.008038,1419.917805
4,5,Phoenix,Arizona,1679243,1615017,1445632,0.019496,1252.474383


In [42]:
population_df.rename(columns={
    "Rank":"rank",
    "Name":"name",
    "State":"state",
    "2018 Population":"2018_population",
    "2016 Population":"2016_population",
    "2010 Census":"2010_population",
    "Change":"change",
    "2018 Density":"2018_density"
}, inplace=True)

In [43]:
population_df

,rank,name,state,2018_population,2016_population,2010_population,change,2018_density
0,1,New York,New York,8580015,8537673,8175133,0.002474,11029.232132
1,2,Los Angeles,California,4030668,3976322,3792621,0.006787,3320.644797
2,3,Chicago,Illinois,2687682,2704958,2695598,-0.003204,4564.612213
3,4,Houston,Texas,2340814,2303482,2099451,0.008038,1419.917805
4,5,Phoenix,Arizona,1679243,1615017,1445632,0.019496,1252.474383
5,6,Philadelphia,Pennsylvania,1573688,1567872,1526006,0.001851,4528.337442
6,7,San Antonio,Texas,1541456,1492510,1327407,0.016133,1291.083266
7,8,San Diego,California,1438060,1406630,1307402,0.011049,1709.720503
8,9,Dallas,Texas,1359133,1317929,1197816,0.015392,1543.608793
9,10,San Jose,California,1030796,1025350,945942,0.002649,2242.032634


In [44]:
population_df.set_index("rank", inplace=True)

In [47]:
population_df.to_sql("population", conn, if_exists="replace", index=True)

## Close connection when complete

In [ ]:
conn.close()